In [1]:
# coding: utf-8

# our all
import numpy as np
import pandas as pd

# usefull pandas settings
pd.set_option('display.max_rows', 45000)
pd.set_option('display.max_columns', 50000)
pd.set_option('display.max_colwidth', 5000)

# for API working and current time
import requests
import datetime

# chess pgn-reading tool
from pgn_parser import pgn, parser

# multistreaming
import threading

# отключим предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')

import matplotlib.pyplot as plt

# own libs
import sys
sys.path.append('//home//roman_vm//my_lib//')

from data_load import *

In [2]:
get_data("""
select max(inserted_at)
from chesscom_players 

""")

,max
0,2020-07-26 20:27:28.284106


In [3]:
get_data("""
select count(distinct target_user)
from chesscom_players 

""")

,count
0,32


In [4]:
get_data("""
select date_part('year', datetime)
        , date_part('month', datetime)
        , count(distinct target_user)
from chesscom_players 
group by date_part('year', datetime)
        , date_part('month', datetime)
order by date_part('year', datetime)
        , date_part('month', datetime)
""")

,date_part,date_part,count
0,1970.0,1.0,7
1,2013.0,1.0,3
2,2013.0,2.0,2
3,2013.0,3.0,2
4,2013.0,4.0,2
5,2013.0,5.0,2
6,2013.0,6.0,2
7,2013.0,7.0,3
8,2013.0,8.0,2
9,2013.0,9.0,3


In [5]:
df=get_data("""

select *
from chesscom_dataset


""")

In [6]:
df['datime'] = df['datime'].astype('datetime64[ns]')
df['year']=df['datime'].dt.year
df['month']=df['datime'].dt.month
df['week']=df['datime'].dt.week
df['day']=df['datime'].dt.day
df['weekday']=df['datime'].dt.weekday
df['hour']=df['datime'].dt.hour

In [10]:
df.sample(3)

,level_0,index,target_user,opponent,datime,event,rules,time_class,time_control,rated,result,score,result_reason,opponent_result_reason,color,rating,opponent_rating,base_elo_forec,eco,game_url,eco_url,inserted_at,game_number,last_game_at,minutes_from_last_game,is_new_session,game_id,session_id,game_ss_number,intransigence_rate,opponent_intransigence_rate,clocker_rate,opponent_clocker_rate,target_opponent_clocker_rate,opponent_opponent_clocker_rate,personal_meets_stat,is_weekend,is_night,year,month,week,day,weekday,hour
11326,11325,0,Rosolimo,Crazy_damage,2017-02-26 16:40:22,Live Chess,chess,blitz,180+2,True,0-1,0.0,resigned,win,w,1662,1837,0.267493,A40,https://www.chess.com/live/game/1970891629,https://www.chess.com/openings/Modern-Defense-Pterodactyl-Beefeater-Variation,2020-07-26 15:29:55.043194,8174,2017-02-26 11:41:38,58.0,1,1,Rosolimo-857,1,0.075,None,0.008,None,0.13,None,0.0,1,0,2017,2,8,26,6,16
5653,5653,0,Rosolimo,damar62,2014-09-26 17:34:28,Live Chess,chess,blitz,300,True,0-1,0.0,resigned,win,w,1513,1544,0.455505,A57,https://www.chess.com/live/game/925057478,https://www.chess.com/openings/Benko-Gambit-Half-Accepted-Zaitsev-System,2020-07-26 15:29:55.043194,3298,2014-09-26 17:25:17,9.0,0,1,Rosolimo-288,19,0.075,None,0.008,None,0.13,None,0.0,0,0,2014,9,39,26,4,17
11041,11040,0,Rosolimo,igo79,2016-12-16 08:39:35,Live Chess,chess,blitz,180+2,True,1-0,1.0,win,resigned,w,1567,1504,0.589684,E10,https://www.chess.com/live/game/1856005638,https://www.chess.com/openings/Indian-Game-East-Indian-Anti-Nimzo-Indian-Variation,2020-07-26 15:29:55.043194,7911,2016-12-16 08:00:16,39.0,1,1,Rosolimo-832,1,0.075,None,0.008,None,0.13,None,1.0,0,0,2016,12,50,16,4,8


In [11]:
len(df)

12281

In [12]:
f_list=[u'event',
       u'rules', u'time_class', u'time_control',  
       u'result_reason', u'opponent_result_reason', 
       u'color', u'rating', u'opponent_rating', u'base_elo_forec', 
       u'eco',
       u'minutes_from_last_game', u'game_ss_number', 
       u'intransigence_rate', u'opponent_intransigence_rate', 
       u'clocker_rate', u'opponent_clocker_rate', 
       u'target_opponent_clocker_rate', u'opponent_opponent_clocker_rate', 
       u'personal_meets_stat', 
       u'is_weekend', u'is_night', 
       u'year', u'month', u'week', u'day', u'weekday', u'hour',
       u'score'
       ]

In [13]:
datatypes=pd.DataFrame(df.dtypes, columns=['type']).reset_index()
f_df=pd.DataFrame(f_list, columns=['is_in_f_lst'])
f_df=f_df.merge(datatypes, 'left', left_on='is_in_f_lst', right_on='index')
f_df=f_df.drop(columns='index')
eho_cols=list(f_df[f_df['type']=='object']['is_in_f_lst'].values)
number_cols=list(f_df[f_df['type']!='object']['is_in_f_lst'].values)

In [14]:
for col in f_df[f_df['type']=='int64']['is_in_f_lst'].values:
    df[col]=df[col].fillna(0)
for col in f_df[f_df['type']=='float64']['is_in_f_lst'].values:
    df[col]=df[col].fillna(0.0)
for col in f_df[f_df['type']=='object']['is_in_f_lst'].values:
    df[col]=df[col].fillna(col+'Unknown')

In [15]:
work_df=df[number_cols]
for col in eho_cols:
    buf_df=pd.get_dummies(df[col])
    work_df=pd.concat([work_df, buf_df], axis=1)

In [16]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

for col in number_cols:
    work_df[col]=scaler.fit_transform(work_df[[col]])

In [18]:
work_df.sample(3)

,rating,opponent_rating,base_elo_forec,minutes_from_last_game,game_ss_number,intransigence_rate,clocker_rate,target_opponent_clocker_rate,personal_meets_stat,is_weekend,is_night,year,month,week,day,weekday,hour,score,Live Chess,chess,blitz,bullet,rapid,120+1,180,180+2,300,300+2,300+3,60,60+1,600,600+2,50move,abandoned,agreed,checkmated,insufficient,repetition,resigned,stalemate,timeout,timevsinsufficient,win,50move,abandoned,agreed,checkmated,insufficient,repetition,resigned,stalemate,timeout,timevsinsufficient,win,b,w,A00,A01,A02,A03,A04,A06,A07,A08,A09,A10,A11,A13,A15,A16,A17,A20,A21,A22,A25,A28,A30,A31,A32,A33,A34,A35,A36,A40,A41,A42,A43,A44,A45,A46,A48,A49,A50,A51,A52,A53,A54,A56,A57,A58,A60,A65,A70,A80,A84,A85,B00,B01,B02,B03,B06,B07,B08,B09,B10,B11,B12,B13,B14,B15,B16,B17,B18,B19,B20,B21,B22,B23,B30,B33,B34,B40,B44,B45,B50,B52,B54,C00,C01,C02,C07,C08,C10,C11,C14,C15,C20,C31,C34,C41,C42,C44,C46,C61,C62,C63,C64,C66,C70,C77,D00,D01,D02,D03,D04,D06,D07,D08,D10,D11,D13,D15,D16,D20,D21,D24,D30,D31,D32,D35,D37,D38,D40,D41,D43,D44,D45,D46,D48,D50,D51,D52,D53,D55,D60,D70,D72,D73,D77,D78,D80,D81,D82,D83,D84,D85,D86,D87,D90,D91,D92,D93,D94,D95,D96,D97,D98,E00,E10,E11,E12,E14,E15,E16,E17,E18,E19,E20,E21,E32,E33,E34,E38,E40,E41,E43,E60,E61,E62,E70,E76,E80,E90,E91,E92,E93,E94,Unknown,opponent_intransigence_rateUnknown,opponent_clocker_rateUnknown,opponent_opponent_clocker_rateUnknown
5402,0.519833,0.596042,0.380225,0.084746,0.118644,0.0,0.0,0.0,0.5,0.0,0.0,0.142857,0.636364,0.634615,0.7,0.666667,0.869565,1.0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1
10037,0.795407,0.599534,0.562703,0.050847,0.254237,0.0,0.0,0.0,1.0,1.0,0.0,0.428571,0.363636,0.384615,0.9,0.833333,0.565217,1.0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1
1499,0.699374,0.607101,0.476570,0.016949,0.101695,0.0,0.0,0.0,0.0,0.0,1.0,0.714286,0.818182,0.788462,0.6,0.666667,0.260870,0.0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1


In [19]:
hold_point='2019-08-01'
test_point='2019-08-01'

In [20]:
work_df['date']=df['datime']
hold_df=work_df[work_df['date']>=hold_point]
ds=work_df[work_df['date']<hold_point]
ds=ds.drop(columns='date')

X=ds.copy().drop(columns='score')
y=ds['score']

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
     X.values, y.values, test_size=0.25, random_state=42)

In [22]:
# строит график, считает метрики
def check(df, x, y1, y2, title):
    from sklearn.metrics import mean_squared_error
    print(title)
    df[[x, y1, y2]].set_index(x).plot(title=title)
    
    print('correlation: ', np.corrcoef(df[y1], df[y2])[0][1])
    
    print('stdev: ', mean_squared_error(df[y1], df[y2]))

In [23]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
lr = LinearRegression().fit(X_train, y_train)
lr_pred=lr.predict(X_test)

print('R^2 train:', lr.score(X_train, y_train))
print('R^2 test:', lr.score(X_test, y_test))
print('Total: ', np.sum(y_test))
print('Total: ', np.sum(lr.predict(X_test)))
print('MAE train', mean_absolute_error(lr.predict(X_train), y_train))
print('MAE test', mean_absolute_error(lr.predict(X_test), y_test))

# plt.plot(range(len(y_test)), y_test, lr.predict(X_test))

('R^2 train:', 1.0)
('R^2 test:', 0.9999999999998236)
('Total: ', 1431.5)
('Total: ', 1431.499964093152)
('MAE train', 3.5462107191853135e-16)
('MAE test', 1.2719393929749448e-08)


In [25]:
import xgboost
from sklearn.metrics import mean_absolute_error, mean_squared_error
xgb = xgboost.XGBRegressor()
xgb.fit(X_train, y_train)
print('R^2 train:', xgb.score(X_train, y_train))
print('R^2 test:', xgb.score(X_test, y_test))
print('Total: ', np.sum(y_test))
print('Total: ', np.sum(xgb.predict(X_test)))
print('MAE train', mean_absolute_error(xgb.predict(X_train), y_train))
print('MAE test', mean_absolute_error(xgb.predict(X_test), y_test))

('R^2 train:', 0.9999999991618593)
('R^2 test:', 0.9999999991618376)
('Total: ', 1431.5)
('Total: ', 1431.4995)
('MAE train', 1.3533999011040185e-05)
('MAE test', 1.3641366458640131e-05)


In [ ]:
xgb_model_abs_testing = list()
xgb_model_abs_training = list()
rng = np.arange(1,151)
for i in rng:
    xgb = xgboost.XGBRegressor(n_estimators=i)
    xgb.fit(X_train, y_train)
    xgb.score(X_test, y_test)
    xgb_model_abs_testing.append(mean_absolute_error(xgb.predict(X_test), y_test))
    xgb_model_abs_training.append(mean_absolute_error(xgb.predict(X_train), y_train))
    
find_df=pd.DataFrame(np.arange(1,151), columns=['n_estimators']).set_index('n_estimators')
find_df['xgb_model_abs_testing_MAE']=xgb_model_abs_testing
find_df['xgb_model_abs_training_MAE']=xgb_model_abs_training

find_df.plot()

stat_best_n_estimators=np.argmin(find_df['xgb_model_abs_testing_MAE'])
ch_n_estimators=stat_best_n_estimators

In [ ]:
xgb_model_abs_testing = list()
xgb_model_abs_training = list()
rng = np.arange(0.05, 0.65, 0.05)
for i in rng:
    xgb = xgboost.XGBRegressor(n_estimators=ch_n_estimators, random_state=17, learning_rate=i)
    xgb.fit(X_train, y_train)
    xgb.score(X_test, y_test)
    xgb_model_abs_testing.append(mean_absolute_error(xgb.predict(X_test), y_test))
    xgb_model_abs_training.append(mean_absolute_error(xgb.predict(X_train), y_train))
    
find_df=pd.DataFrame(rng, columns=['learning_rate']).set_index('learning_rate')
find_df['xgb_model_abs_testing_MAE']=xgb_model_abs_testing
find_df['xgb_model_abs_training_MAE']=xgb_model_abs_training

find_df.plot()

stat_best_learning_rate=np.argmin(find_df['xgb_model_abs_testing_MAE'])
ch_learning_rate=0.15

In [ ]:
xgb_model_abs_testing = list()
xgb_model_abs_training = list()
rng = np.arange(1, 11)
for i in rng:
    xgb = xgboost.XGBRegressor(n_estimators=ch_n_estimators, random_state=17, learning_rate=ch_learning_rate, max_depth=i)
    xgb.fit(X_train, y_train)
    xgb.score(X_test, y_test)
    xgb_model_abs_testing.append(mean_absolute_error(xgb.predict(X_test), y_test))
    xgb_model_abs_training.append(mean_absolute_error(xgb.predict(X_train), y_train))
    
find_df=pd.DataFrame(rng, columns=['max_depth']).set_index('max_depth')
find_df['xgb_model_abs_testing_MAE']=xgb_model_abs_testing
find_df['xgb_model_abs_training_MAE']=xgb_model_abs_training

find_df.plot()

stat_best_max_depth=np.argmin(find_df['xgb_model_abs_testing_MAE'])
ch_max_depth=2

In [ ]:
ch_max_depth=2

In [ ]:
import xgboost
from sklearn.metrics import mean_absolute_error, mean_squared_error
xgb = xgboost.XGBRegressor(n_estimators=ch_n_estimators, random_state=17, learning_rate=ch_learning_rate, max_depth=ch_max_depth)
xgb.fit(X_train, y_train)
print('R^2 train:', xgb.score(X_train, y_train))
print('R^2 test:', xgb.score(X_test, y_test))
print('Total: ', np.sum(y_test))
print('Total: ', np.sum(xgb.predict(X_test)))
print('MAE train', mean_absolute_error(xgb.predict(X_train), y_train))
print('MAE test', mean_absolute_error(xgb.predict(X_test), y_test))

In [ ]:
res_test=pd.DataFrame(X_test, columns=X.columns)
res_test['fact']=y_test
res_test['predict']=xgb.predict(X_test)


sh=res_test.sample(150).reset_index()
sh['ind']=sh.index

In [ ]:
check(sh, 'ind', 'fact', 'predict', 'gg')

In [ ]:
features_df = pd.DataFrame(data=xgb.feature_importances_.reshape(1, -1), columns=X.columns).sort_values(axis=1, by=[0], ascending=False).T

In [ ]:
features_df

In [ ]:
res_test.groupby('month').mean()[['fact']]

In [ ]:
res_test.groupby('hour').mean()[['fact']]